In [ ]:
#Possible features
#difference between character removal

In [1]:
import pandas as pd 
import numpy as np

import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('punkt')
from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import CountVectorizer

from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize
import textstat
import spacy
nlp = spacy.load("en_core_web_sm")

from textblob import TextBlob

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\brand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df=pd.read_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\Data_merged.parquet")
print(df.info())
df=df.sample(frac=1,random_state=42)#.head(20000)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61719 entries, 0 to 61718
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   61719 non-null  object 
 1   title   61719 non-null  object 
 2   text    61719 non-null  object 
 3   label   61719 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.9+ MB
None


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61719 entries, 46996 to 56422
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   61719 non-null  object 
 1   title   61719 non-null  object 
 2   text    61719 non-null  object 
 3   label   61719 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.4+ MB


In [4]:
# Vectorized function to calculate word count
word_count_func = np.vectorize(lambda text: len(word_tokenize(text)))
df['word_count'] = word_count_func(df['text'])


In [5]:
'''
# Vectorized function to calculate average word length
avg_word_length_func = np.vectorize(lambda text: np.mean([len(word) for word in word_tokenize(text)]))
df['avg_word_length'] = avg_word_length_func(df['text'])
'''

"\n# Vectorized function to calculate average word length\navg_word_length_func = np.vectorize(lambda text: np.mean([len(word) for word in word_tokenize(text)]))\ndf['avg_word_length'] = avg_word_length_func(df['text'])\n"

In [6]:
# Vectorized function to calculate average sentence length
avg_sentence_length_func = np.vectorize(lambda text: np.mean([len(word_tokenize(sentence)) for sentence in sent_tokenize(text)]))
df['avg_sentence_length'] = avg_sentence_length_func(df['text'])


D:\Programs\envs\FakeNewsResearch\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\Programs\envs\FakeNewsResearch\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [7]:

# Vectorized function to calculate the frequency of punctuation marks
vectorized_calculate_punctuation_frequency = np.vectorize(lambda text: len(re.findall(r'[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', text)))

# Vectorized function to calculate the frequency of capitalized words or phrases
vectorized_calculate_capitalization_frequency = np.vectorize(lambda text: sum(1 for word in text.split() if word.isupper()))

# Apply the vectorized functions to create 'punctuation_frequency' and 'capitalization_frequency' columns
df['text_punctuation_frequency'] = vectorized_calculate_punctuation_frequency(df['text'])
df['text_capitalization_frequency'] = vectorized_calculate_capitalization_frequency(df['text'])

df['title_punctuation_frequency'] = vectorized_calculate_punctuation_frequency(df['title'])
df['title_capitalization_frequency'] = vectorized_calculate_capitalization_frequency(df['title'])


In [8]:

# Display the DataFrame
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 61719 entries, 46996 to 56422
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   index                           61719 non-null  object 
 1   title                           61719 non-null  object 
 2   text                            61719 non-null  object 
 3   label                           61719 non-null  float64
 4   word_count                      61719 non-null  int32  
 5   avg_sentence_length             61716 non-null  float64
 6   text_punctuation_frequency      61719 non-null  int32  
 7   text_capitalization_frequency   61719 non-null  int32  
 8   title_punctuation_frequency     61719 non-null  int32  
 9   title_capitalization_frequency  61719 non-null  int32  
dtypes: float64(2), int32(5), object(3)
memory usage: 4.0+ MB


In [9]:
df['title_before_character_removal']=df['title'].str.len()
df['text_before_character_removal']=df['text'].str.len()

def remove_non_characters(text):
    # Remove non-character characters using regular expressions
    cleaned_text = text.str.replace(r'[^a-zA-Z\s]', '', regex=True)
    return cleaned_text

# Apply the function to 'title' column
df['title'] = remove_non_characters(df['title'])

# Apply the function to 'text' column
df['text'] = remove_non_characters(df['text'])

# Convert to lowercase
df['title'] = df['title'].str.lower()
df['text'] = df['text'].str.lower()

df['title_after_character_removal']=df['title'].str.len()
df['text_after_character_removal']=df['text'].str.len()

df.describe()

,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,title_punctuation_frequency,title_capitalization_frequency,title_before_character_removal,text_before_character_removal,title_after_character_removal,text_after_character_removal
count,61719.000000,61719.000000,61716.000000,61719.0,61719.000000,61719.0,61719.000000,61719.000000,61719.000000,61719.000000,61719.000000
mean,0.442068,632.737083,30.148830,0.0,9.301026,0.0,1.111343,75.249486,3335.958603,72.514639,3213.814579
std,0.496637,714.952566,11.876844,0.0,15.780880,0.0,2.044250,22.544133,3643.617011,21.822781,3471.998371
min,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,1.000000
25%,0.000000,282.000000,24.279767,0.0,3.000000,0.0,0.000000,61.000000,1512.000000,60.000000,1452.500000
50%,0.000000,461.000000,28.470588,0.0,6.000000,0.0,0.000000,72.000000,2490.000000,70.000000,2407.000000
75%,1.000000,776.000000,33.500000,0.0,11.000000,0.0,1.000000,86.000000,4132.000000,83.000000,3982.500000
max,1.000000,28009.000000,516.571429,0.0,909.000000,0.0,24.000000,456.000000,142961.000000,443.000000,129415.000000


In [10]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Tokenize the text into individual words
    words = nltk.word_tokenize(text)
    
    # Remove stop words
    words = [word for word in words if word.lower() not in stop_words]
    
    # Perform stemming on each word
    words = [stemmer.stem(word) for word in words]
    
    # Join the processed words back into a single string
    processed_text = ' '.join(words)
    
    return processed_text

df['title_processed'] = ''
df['text_processed'] = ''

def process_row(row):
    title = row['title']
    text = row['text']
    
    processed_title = preprocess_text(title)
    processed_text = preprocess_text(text)

    
    return pd.Series([processed_title, processed_text])

df[['title_processed', 'text_processed']] = df.apply(process_row, axis=1)

In [11]:
calculate_subjectivity_vectorized = np.vectorize(lambda text: TextBlob(text).sentiment.subjectivity, otypes=[float])

# Apply the vectorized function to create a 'subjectivity' column
df['subjectivity_text'] = calculate_subjectivity_vectorized(df['text_processed'])
df['subjectivity_title'] = calculate_subjectivity_vectorized(df['title_processed'])


In [12]:
sia = SentimentIntensityAnalyzer()

# Create progress bars for title and text sentiment analysis
df['title_sentiment'] = [sia.polarity_scores(text)['compound'] for text in df['title_processed']]


# Apply sentiment analysis to 'text_processed' column
df['text_sentiment'] = [sia.polarity_scores(text)['compound'] for text in df['text_processed']]



df.head()

,index,title,text,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,title_punctuation_frequency,title_capitalization_frequency,title_before_character_removal,text_before_character_removal,title_after_character_removal,text_after_character_removal,title_processed,text_processed,subjectivity_text,subjectivity_title,title_sentiment,text_sentiment
46996,b'52840',eu envoys start discussion on ties with london...,brussels reuters european union nations excep...,0.0,759,27.107143,0,22,0,1,59,3997,59,3915,eu envoy start discuss tie london brexit,brussel reuter european union nation except br...,0.348177,0.0,0.0000,0.9921
59626,b'69362',watch morning joe hosts urge republicans to a...,if republicans listen to this sage advice dona...,1.0,508,46.181818,0,1,0,2,96,2795,94,2730,watch morn joe host urg republican abandon tru...,republican listen sage advic donald trump woul...,0.505126,0.0,-0.4404,-0.9914
49392,b'55854',spains rajoy wants to work with other parties ...,madrid reuters the spanish government is comm...,0.0,106,35.333333,0,1,0,0,66,630,65,617,spain rajoy want work parti catalan question,madrid reuter spanish govern commit seek joint...,0.000000,0.0,0.0772,0.7506
48873,b'55206',trump was asked what he has in common with his...,\nrepublican nominee donald j trump has been...,1.0,435,24.166667,0,3,0,1,77,2070,76,1971,trump ask common daughter answer creepi,republican nomine donald j trump vocal creepi ...,0.467216,0.5,0.0000,0.6705
17631,b'18280',clinton knocks trump for cheering housing bubb...,washington reuters democrat hillary clinton s...,0.0,924,27.176471,0,11,0,0,54,4819,54,4642,clinton knock trump cheer hous bubbl burst,washington reuter democrat hillari clinton see...,0.513441,0.0,0.5106,0.1280


In [13]:
def calculate_reading_ease(text):
    if isinstance(text, float):
        return 0  # Return 0 for float values
    
    # Calculate the reading ease score
    reading_ease = textstat.flesch_reading_ease(text)
    
    return reading_ease

vectorized_calculate_reading_ease = np.vectorize(calculate_reading_ease, otypes=[float])

# Assuming you have a DataFrame 'df' with 'title_processed' and 'text_processed' columns
df['title_reading_ease'] = vectorized_calculate_reading_ease(df['title_processed'])
df['text_reading_ease'] = vectorized_calculate_reading_ease(df['text_processed'])

df.head()

,index,title,text,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,title_punctuation_frequency,title_capitalization_frequency,...,title_after_character_removal,text_after_character_removal,title_processed,text_processed,subjectivity_text,subjectivity_title,title_sentiment,text_sentiment,title_reading_ease,text_reading_ease
46996,b'52840',eu envoys start discussion on ties with london...,brussels reuters european union nations excep...,0.0,759,27.107143,0,22,0,1,...,59,3915,eu envoy start discuss tie london brexit,brussel reuter european union nation except br...,0.348177,0.0,0.0000,0.9921,64.37,-306.79
59626,b'69362',watch morning joe hosts urge republicans to a...,if republicans listen to this sage advice dona...,1.0,508,46.181818,0,1,0,2,...,94,2730,watch morn joe host urg republican abandon tru...,republican listen sage advic donald trump woul...,0.505126,0.0,-0.4404,-0.9914,50.84,-178.56
49392,b'55854',spains rajoy wants to work with other parties ...,madrid reuters the spanish government is comm...,0.0,106,35.333333,0,1,0,0,...,65,617,spain rajoy want work parti catalan question,madrid reuter spanish govern commit seek joint...,0.000000,0.0,0.0772,0.7506,64.37,-10.75
48873,b'55206',trump was asked what he has in common with his...,\nrepublican nominee donald j trump has been...,1.0,435,24.166667,0,3,0,1,...,76,1971,trump ask common daughter answer creepi,republican nomine donald j trump vocal creepi ...,0.467216,0.5,0.0000,0.6705,73.85,-107.85
17631,b'18280',clinton knocks trump for cheering housing bubb...,washington reuters democrat hillary clinton s...,0.0,924,27.176471,0,11,0,0,...,54,4642,clinton knock trump cheer hous bubbl burst,washington reuter democrat hillari clinton see...,0.513441,0.0,0.5106,0.1280,89.75,-402.54


In [14]:
def calculate_lexical_diversity(text):
    if isinstance(text, float):
        return 0  # Return 0 for float values
    
    # Tokenize the text into individual words
    words = text.split()
    
    # Calculate the lexical diversity score
    if len(words) > 0:
        lexical_diversity = len(set(words)) / len(words)
    else:
        lexical_diversity = 0
    
    return lexical_diversity

# Define a vectorized version of calculate_lexical_diversity
vectorized_calculate_lexical_diversity = np.vectorize(calculate_lexical_diversity, otypes=[float])

# Assuming you have a DataFrame 'df_12' with 'title_processed' and 'text_processed' columns

df['text_lexical_diversity'] = vectorized_calculate_lexical_diversity(df['text_processed'])
df['title_lexical_diversity'] = vectorized_calculate_lexical_diversity(df['title_processed'])

df.head()

,index,title,text,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,title_punctuation_frequency,title_capitalization_frequency,...,title_processed,text_processed,subjectivity_text,subjectivity_title,title_sentiment,text_sentiment,title_reading_ease,text_reading_ease,text_lexical_diversity,title_lexical_diversity
46996,b'52840',eu envoys start discussion on ties with london...,brussels reuters european union nations excep...,0.0,759,27.107143,0,22,0,1,...,eu envoy start discuss tie london brexit,brussel reuter european union nation except br...,0.348177,0.0,0.0000,0.9921,64.37,-306.79,0.538058,1.0
59626,b'69362',watch morning joe hosts urge republicans to a...,if republicans listen to this sage advice dona...,1.0,508,46.181818,0,1,0,2,...,watch morn joe host urg republican abandon tru...,republican listen sage advic donald trump woul...,0.505126,0.0,-0.4404,-0.9914,50.84,-178.56,0.722689,1.0
49392,b'55854',spains rajoy wants to work with other parties ...,madrid reuters the spanish government is comm...,0.0,106,35.333333,0,1,0,0,...,spain rajoy want work parti catalan question,madrid reuter spanish govern commit seek joint...,0.000000,0.0,0.0772,0.7506,64.37,-10.75,0.803571,1.0
48873,b'55206',trump was asked what he has in common with his...,\nrepublican nominee donald j trump has been...,1.0,435,24.166667,0,3,0,1,...,trump ask common daughter answer creepi,republican nomine donald j trump vocal creepi ...,0.467216,0.5,0.0000,0.6705,73.85,-107.85,0.767568,1.0
17631,b'18280',clinton knocks trump for cheering housing bubb...,washington reuters democrat hillary clinton s...,0.0,924,27.176471,0,11,0,0,...,clinton knock trump cheer hous bubbl burst,washington reuter democrat hillari clinton see...,0.513441,0.0,0.5106,0.1280,89.75,-402.54,0.608137,1.0


In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Assuming you have a DataFrame 'df' with a 'text_processed' column containing preprocessed text

# Tokenize the text, convert to lowercase, and remove punctuation
#df['text_processed_ve'] = df['text_processed'].apply(lambda text: " ".join(text.split()[:5]).lower())

# Initialize CountVectorizer with optional preprocessing options
vectorizer = CountVectorizer(
    stop_words='english',  # Remove common stop words
    max_features=1000,     # Limit the number of features to reduce dimensionality
    binary=True             # Use binary encoding (1 if word is present, 0 otherwise)
)

# Vectorize the text
X = vectorizer.fit_transform(df['text_processed'])

# Create a DataFrame with the BoW representation
bag_of_words_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Reset the index of df
df.reset_index(drop=True, inplace=True)

# Add a prefix to the BoW columns
new_column_names = ["BOW_" + col for col in bag_of_words_df.columns]
bag_of_words_df.columns = new_column_names

# Save the BoW DataFrame to a Parquet file
bag_of_words_df.to_parquet("BOW.parquet")

# Concatenate df and bag_of_words_df
concatenated_df = pd.concat([df, bag_of_words_df], axis=1)


In [16]:
#Best of 5 text
bag_of_words_df.shape
bag_of_words_df.head()
#compund words

,BOW_abil,BOW_abl,BOW_absolut,BOW_abus,BOW_accept,BOW_access,BOW_accord,BOW_account,BOW_accus,BOW_achiev,...,BOW_worth,BOW_write,BOW_written,BOW_wrong,BOW_wrote,BOW_ye,BOW_year,BOW_yearold,BOW_york,BOW_young
0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
'''
import spacy

nlp = spacy.load("en_core_web_sm")

# Assuming you have a DataFrame 'df' with a 'text_processed' column
df['compound_words'] = ""

for index, row in df.iterrows():
    text = row['text_processed']
    
    # Process the text using spaCy
    doc = nlp(text)
    
    # Extract named entities (compound words)
    compound_words = [entity.text for entity in doc.ents if len(entity.text.split()) > 1]
    
    # Store the results in the DataFrame
    df.at[index, 'compound_words'] = ', '.join(compound_words)

'''

'\nimport spacy\n\nnlp = spacy.load("en_core_web_sm")\n\n# Assuming you have a DataFrame \'df\' with a \'text_processed\' column\ndf[\'compound_words\'] = ""\n\nfor index, row in df.iterrows():\n    text = row[\'text_processed\']\n    \n    # Process the text using spaCy\n    doc = nlp(text)\n    \n    # Extract named entities (compound words)\n    compound_words = [entity.text for entity in doc.ents if len(entity.text.split()) > 1]\n    \n    # Store the results in the DataFrame\n    df.at[index, \'compound_words\'] = \', \'.join(compound_words)\n\n'

In [18]:
#print(df['compound_words'].head(1))

In [19]:

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61719 entries, 0 to 61718
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   index                           61719 non-null  object 
 1   title                           61719 non-null  object 
 2   text                            61719 non-null  object 
 3   label                           61719 non-null  float64
 4   word_count                      61719 non-null  int32  
 5   avg_sentence_length             61716 non-null  float64
 6   text_punctuation_frequency      61719 non-null  int32  
 7   text_capitalization_frequency   61719 non-null  int32  
 8   title_punctuation_frequency     61719 non-null  int32  
 9   title_capitalization_frequency  61719 non-null  int32  
 10  title_before_character_removal  61719 non-null  int64  
 11  text_before_character_removal   61719 non-null  int64  
 12  title_after_character_removal   

In [20]:
#df.dropna(axis=0,inplace=True)

df.to_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\ForModeling.parquet")

concatenated_df.to_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\ForModeling_BagofWords.parquet")

df.to_csv(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Visualisation\Visualize.csv")



In [21]:
concatenated_df

,index,title,text,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,title_punctuation_frequency,title_capitalization_frequency,...,BOW_worth,BOW_write,BOW_written,BOW_wrong,BOW_wrote,BOW_ye,BOW_year,BOW_yearold,BOW_york,BOW_young
0,b'52840',eu envoys start discussion on ties with london...,brussels reuters european union nations excep...,0.0,759,27.107143,0,22,0,1,...,0,0,0,0,0,0,1,0,0,0
1,b'69362',watch morning joe hosts urge republicans to a...,if republicans listen to this sage advice dona...,1.0,508,46.181818,0,1,0,2,...,0,0,0,0,0,0,0,0,0,0
2,b'55854',spains rajoy wants to work with other parties ...,madrid reuters the spanish government is comm...,0.0,106,35.333333,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,b'55206',trump was asked what he has in common with his...,\nrepublican nominee donald j trump has been...,1.0,435,24.166667,0,3,0,1,...,0,0,0,0,0,0,1,0,0,0
4,b'18280',clinton knocks trump for cheering housing bubb...,washington reuters democrat hillary clinton s...,0.0,924,27.176471,0,11,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61714,b'62282',north korea says rockets to us inevitable as u...,united nationswashington reuters north korea ...,0.0,899,28.093750,0,21,0,2,...,0,0,0,0,0,0,1,0,0,0
61715,b'41951',peru raises cost of postfloods rebuilding to n...,lima reuters previously uncalculated damage c...,0.0,223,27.875000,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
61716,b'546',trump clinton tension seeps into jokes at annu...,new york reuters us presidential campaign ten...,0.0,600,33.333333,0,8,0,0,...,0,0,0,0,0,0,0,0,1,0
61717,b'16277',iran calls on muslim nations to step up effort...,istanbul reuters iranian president hassan rou...,0.0,85,28.333333,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df

,index,title,text,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,title_punctuation_frequency,title_capitalization_frequency,...,title_processed,text_processed,subjectivity_text,subjectivity_title,title_sentiment,text_sentiment,title_reading_ease,text_reading_ease,text_lexical_diversity,title_lexical_diversity
0,b'52840',eu envoys start discussion on ties with london...,brussels reuters european union nations excep...,0.0,759,27.107143,0,22,0,1,...,eu envoy start discuss tie london brexit,brussel reuter european union nation except br...,0.348177,0.000000,0.0000,0.9921,64.37,-306.79,0.538058,1.000000
1,b'69362',watch morning joe hosts urge republicans to a...,if republicans listen to this sage advice dona...,1.0,508,46.181818,0,1,0,2,...,watch morn joe host urg republican abandon tru...,republican listen sage advic donald trump woul...,0.505126,0.000000,-0.4404,-0.9914,50.84,-178.56,0.722689,1.000000
2,b'55854',spains rajoy wants to work with other parties ...,madrid reuters the spanish government is comm...,0.0,106,35.333333,0,1,0,0,...,spain rajoy want work parti catalan question,madrid reuter spanish govern commit seek joint...,0.000000,0.000000,0.0772,0.7506,64.37,-10.75,0.803571,1.000000
3,b'55206',trump was asked what he has in common with his...,\nrepublican nominee donald j trump has been...,1.0,435,24.166667,0,3,0,1,...,trump ask common daughter answer creepi,republican nomine donald j trump vocal creepi ...,0.467216,0.500000,0.0000,0.6705,73.85,-107.85,0.767568,1.000000
4,b'18280',clinton knocks trump for cheering housing bubb...,washington reuters democrat hillary clinton s...,0.0,924,27.176471,0,11,0,0,...,clinton knock trump cheer hous bubbl burst,washington reuter democrat hillari clinton see...,0.513441,0.000000,0.5106,0.1280,89.75,-402.54,0.608137,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61714,b'62282',north korea says rockets to us inevitable as u...,united nationswashington reuters north korea ...,0.0,899,28.093750,0,21,0,2,...,north korea say rocket us inevit us bomber fli...,unit nationswashington reuter north korea said...,0.394075,0.000000,0.0000,-0.9944,76.22,-445.17,0.530452,0.833333
61715,b'41951',peru raises cost of postfloods rebuilding to n...,lima reuters previously uncalculated damage c...,0.0,223,27.875000,0,1,0,0,...,peru rais cost postflood rebuild nearli billion,lima reuter previous uncalcul damag caus sever...,0.442016,0.000000,0.0000,0.0026,55.91,-57.44,0.779528,1.000000
61716,b'546',trump clinton tension seeps into jokes at annu...,new york reuters us presidential campaign ten...,0.0,600,33.333333,0,8,0,0,...,trump clinton tension seep joke annual chariti...,new york reuter us presidenti campaign tension...,0.402622,0.000000,-0.0258,0.8020,37.98,-216.79,0.707746,1.000000
61717,b'16277',iran calls on muslim nations to step up effort...,istanbul reuters iranian president hassan rou...,0.0,85,28.333333,0,1,0,0,...,iran call muslim nation step effort trump jeru...,istanbul reuter iranian presid hassan rouhani ...,0.301190,0.000000,0.0000,-0.3182,79.26,19.71,0.823529,1.000000
